In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import data.preprocessing as prep
from model.model import Device_Model, generate_weight

from math import floor
from sklearn.utils import shuffle
from random import randint
from tensorflow.keras import datasets
from keras.utils import to_categorical
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

C:\Users\KevinLBJ\Anaconda3\envs\fl\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


KeyboardInterrupt: 

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

with tf.device('/device:GPU:0'):
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)

In [ ]:
# Load CIFAR-10 data
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

train_labels = train_labels.reshape(len(train_labels),)
train_label = to_categorical(train_labels)
test_label = to_categorical(test_labels)

In [ ]:
def prepare_for_training_data_selected_random(device_num, device_data_amount):
    temp_amount = round(device_data_amount/10)
    image, label = train_images, train_labels
    
    temp_image, temp_label = image[label == 0], label[label == 0]
    shuffle(temp_image, temp_label, random_state=device_num)
    
    s0 = [temp_image[:temp_amount], temp_label[:temp_amount]]
    
    for classes in range(1, 10):
        temp_image, temp_label = image[label == classes], label[label == classes]
        shuffle(temp_image, temp_label, random_state=device_num * classes)

        s1 = [temp_image[:temp_amount], temp_label[:temp_amount]]
        s0 = [np.append(s0[0], s1[0], axis=0), np.append(s0[1], s1[1])]
    
    s0[0], s0[1] = shuffle(s0[0], s0[1], random_state=randint(0, device_num))

    return s0[0], to_categorical(s0[1], 10)

In [ ]:
def step_decay(epoch):
    epoch = _ + epo
    # initial_lrate = 1.0 # no longer needed
    drop = 0.99
    epochs_drop = 1.0
    
    lrate = 0.2 * pow(drop, floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:
# adjust parameters of the model
callback = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [ ]:
augment = ImageDataGenerator(preprocessing_function=prep.preprocessing_for_training)
history_02 = {}

In [ ]:
# def define_model():
#     return tf.keras.Sequential([
#         layers.Conv2D(64, (5, 5), padding='same', activation='relu', input_shape=(24, 24, 3)),
#         layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
#         layers.BatchNormalization(),
#         layers.Conv2D(64, (5,5), padding='same', activation='relu'),
#         layers.MaxPool2D(pool_size=3, strides=2, padding='same'),
#         layers.BatchNormalization(),
#         layers.Flatten(),
#         layers.Dense(384, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(192, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(10, activation='softmax')
#     ])

In [ ]:
def prepare_for_training_data_selected_random_range(device_num, data_range):
    image, label = train_images, train_labels

    if data_range[0] < data_range[1]:
        s0 = [image[label == 0][data_range[0]:data_range[1]], label[label == 0][data_range[0]:data_range[1]]]

        for classes in range(1, 10):
            s1 = [image[label == classes][data_range[0]:data_range[1]], label[label == classes][data_range[0]:data_range[1]]]
            s0 = [np.append(s0[0], s1[0], axis=0), np.append(s0[1], s1[1])]

    else:
        s1 = [image[label == 0][data_range[0]:], label[label == 0][data_range[0]:]]
        a = image[label == 0][:data_range[1]]
        a_label = label[label == 0][:data_range[1]]

        s0 = [np.append(a, s1[0], axis=0), np.append(a_label, s1[1])]

        for classes in range(1, 10):
            s1 = [image[label == classes][data_range[0]:], label[label == classes][data_range[0]:]]
            a = image[label == 0][:data_range[1]]
            a_label = label[label == 0][:data_range[1]]
            s1 = [np.append(a, s1[0], axis=0), np.append(a_label, s1[1])]

            s0 = [np.append(s0[0], s1[0], axis=0), np.append(s0[1], s1[1])]

    for _ in range(20):
        s0[0], s0[1] = shuffle(s0[0], s0[1], random_state=randint(0, device_num))
        
    return s0[0], to_categorical(s0[1], 10)

In [ ]:
# class Device_Model():
#     def __init__(self, number):
#         self.number = number
#         self.weight = define_model()
#         self.weight.compile(optimizer='sgd',
#                                   loss='categorical_crossentropy',
#                                   metrics=['accuracy'])
#         self.weight.load_weights('ini_cifar10.h5')
        
#         self.client = {}
#         self.history = {}
#         self.first = {}
#         self.master_first = True
#         self.weight_hat = define_model()
#         self.weight_q = define_model()
        
#         self.weight_list = []
        
#     def add_client(self, client, degree):
#         if client not in self.client:
#             self.client[client] = {}
#             self.client[client]['weight'] = define_model()
#             self.client[client]['degree'] = degree
#             self.first[client] = True
#         else:
#             print(client, "has already existed in client dictionary.")
        
#     def add_client_list(self, client, client_dictionary):
#         for cli in client:
#             if cli not in self.client:
#                 self.client[cli] = {}
#                 self.client[cli]['weight'] = define_model()
#                 self.client[cli]['degree'] = len(client_dictionary[cli])
#                 self.first[cli] = True
#             else:
#                 print(cli, "has already existed in client dictionary.")
        
#     def set_client_weight(self, client, value):
#         if client in self.client:
#             if self.first[client] == True:
#                 self.first[client] = False
#                 self.client[client]['weight'].set_weights(value.get_weights())
#             else:
#                 for layer in range(len(self.weight.layers)):
#                     self.client[client]['weight'].layers[layer].set_weights(
#                         np.add(value.layers[layer].get_weights(),
#                           self.client[client]['weight'].layers[layer].get_weights()))
#         else:
#             print(client, "doesn't exist in client dictionary.")
    
#     def update_own_q(self):
#         if self.master_first == True:
#             self.master_first = False
#             self.weight_hat.set_weights(self.weight_q.get_weights())
#         else:
#             for layer in range(len(self.weight.layers)):
#                 self.weight_hat.layers[layer].set_weights(
#                     np.add(self.weight_q.layers[layer].get_weights(), 
#                       self.weight_hat.layers[layer].get_weights()))
            
#     def count_q(self):
#         if self.master_first == True:
#             self.weight_q.set_weights(self.weight.get_weights())
#         else:
#             for layer in range(len(self.weight.layers)):
#                 self.weight_q.layers[layer].set_weights(
#                     np.subtract(self.weight.layers[layer].get_weights(), 
#                     self.weight_hat.layers[layer].get_weights()))
    
#     def update_parameter(self):
#         if self.master_first == False:
#             client_list = [x for x in self.client]
            
#             temp = {}
            
#             for device in client_list:
#                 client_weight = self.weight_list[device] * 0.5
                
#                 if client_list.index(device) == 0:
#                     for layer in range(len(self.weight.layers)):
#                         temp[layer] = np.subtract(self.client[device]['weight'].layers[layer].get_weights(), 
#                                     self.weight_hat.layers[layer].get_weights())
#                 elif client_list.index(device) == len(client_list) - 1:
#                     for layer in range(len(self.weight.layers)):
#                         self.weight.layers[layer].set_weights(
#                             np.add(self.weight.layers[layer].get_weights(),
#                               np.multiply(np.add(temp[layer], 
#                                   np.subtract(self.client[device]['weight'].layers[layer].get_weights(), 
#                                     self.weight_hat.layers[layer].get_weights())), client_weight)))
                                
#                 else:
#                     for layer in range(len(self.weight.layers)):
#                         temp[layer] = np.add(temp[layer], 
#                                         np.subtract(self.client[device]['weight'].layers[layer].get_weights(), 
#                                           self.weight_hat.layers[layer].get_weights()))
                        
#     def set_weight(self, device_dictionary):
#         self.weight_list = generate_weight(self.number, device_dictionary)

In [ ]:
def sort_dictionary(dic):
    temp = sorted(dic.keys())
    temp_dic = {}
    
    for key in temp:
        temp_dic[key] = dic[key]
    
    return temp_dic

In [ ]:
# def generate_weight(node, device_client_dic):
#     temp = []
    
#     for j in range(num_node):
#         if j in device_client_dic[node]:
#             temp.append(1/(max(len(device_client_dic[node]), len(device_client_dic[j])) + 1))
#         else:
#             temp.append(0)
            
#     temp_v = 0    
#     for j in device_client_dic[node]:
#         temp_v += temp[j]
        
#     temp[node] = 1 - temp_v
    
#     return temp

In [ ]:
device_client_dic = {}
alg_selected_device = []

total_demand = 50000
num_node = 15

for n in range(num_node):
    node_list = [x for x in range(num_node)]

    node_list.remove(n)

    device_client_dic[n] = node_list

device_client_list = [x for x in device_client_dic]
        
for num in range(num_node):
    alg_selected_device.append(total_demand/num_node)

In [ ]:
num_device = len(alg_selected_device)
num_center_epoch = 1
num_local_epoch = 5
num_round = 10000
center_batch_size = 64
local_batch_size = 64

show = 0

device_train_data = {}
device_test_data = {}

for _ in range(num_round):
    print("\033[1m" + "Round: " + str(_) + '\033[0m')
    start_with = 0
    end_with = 0

    for device in device_client_dic:
        if(_ == 0):
            #Define an estimator model
            #Initialize every device (e.g., all devices are initialized with same parameters)
            locals()['model_{}'.format(device)] = Device_Model(device)
            locals()['model_{}'.format(device)].history['val_loss'] = [2.3840]
            locals()['model_{}'.format(device)].history['val_acc'] = [0.0976]

            locals()['model_{}'.format(device)].add_client_list(device_client_dic[device], device_client_dic)
            locals()['model_{}'.format(device)].set_weight(device_client_dic, num_node)

            temp_arange = []
            temp_amount = int(alg_selected_device[device]/10) + 10

            end_with += temp_amount
            end_with %= 5000
            temp_arange = [start_with, end_with]

            train_image_temp, train_label_temp = prepare_for_training_data_selected_random_range(device, temp_arange)
            start_with = end_with
            start_with %= 5000

            device_train_data[device] = [train_image_temp, train_label_temp]


        #Local training on each device 
        for epo in range(num_local_epoch):
            train_image_crop = np.stack([prep.random_crop(device_train_data[device][0][i], 24, 24) for i in range(len(device_train_data[device][0]))], axis=0)

            for random_ in range(10):
                train_new_image, train_new_label = shuffle(train_image_crop, 
                                                           device_train_data[device][1], 
                                                           random_state=randint(0, train_image_crop.shape[0]))

            history_temp = locals()['model_{}'.format(device)].weight.fit_generator(
                augment.flow(train_new_image, train_new_label, batch_size=local_batch_size), 
                epochs=1, 
                callbacks=[callback],
                verbose=show)

        # Update from x^(t+1/2) to x^(t+1)   (line 4)
        locals()['model_{}'.format(device)].update_parameter()

        # Count weight_q (line 5)
        locals()['model_{}'.format(device)].count_q()

    # Update all weight_hat for each device (line 6, 7)
    for device in device_client_dic:
        for dev in device_client_dic[device]:
            locals()['model_{}'.format(device)].set_client_weight(dev, locals()['model_{}'.format(dev)].weight_q)

    # Update own weight_hat for each device (line 8)
    for device in device_client_dic:
        locals()['model_{}'.format(device)].update_own_q()

    for device in device_client_dic:
        #Evaluate with new weight
        test_d = np.stack([prep.preprocessing_for_testing(test_images[i]) for i in range(10000)], axis=0)

        test_new_image, test_new_label = shuffle(test_d, test_label, 
                                                 random_state=randint(1, train_images.shape[0]))

        history_temp = locals()['model_{}'.format(device)].weight.evaluate(test_new_image, 
                                                                           test_new_label, 
                                                                           batch_size=64,
                                                                           verbose=show)

        #Record each round accuracy and loss for every device
        locals()['model_{}'.format(device)].history['val_loss'].append(history_temp[0])
        locals()['model_{}'.format(device)].history['val_acc'].append(history_temp[1])
        print(str(_), device, " Result: ", str(history_temp[1]))

#     if _ % 100 == 0 or _ == num_round - 1:
#         with open('Federated_Learning_Data/15-nodes/fully_result_matrix_05.txt', 'w+') as f:
#             for device in device_client_dic:
#                 f.write(str(device))
#                 f.write(str(locals()['model_{}'.format(device)].history['val_acc']))
#                 f.write(str(locals()['model_{}'.format(device)].history['val_loss']))
#                 f.write('\n')